In [74]:
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
channel1 = 'Scikitlearn'
channel2 = 'Angular'

In [4]:
df_issue = pd.read_csv(channel2+'/'+channel2+'_issue_tags.csv')

In [5]:
df_issue.head()

,Id,asker,time,content,lines,tags,content_stem
0,1,54c89b1cdb8155e6700f2ac3,2015-03-07 21:41,Awesome!\n,1,[],awesom!\n
1,2,54e6ee3b15522ed4b3dc3c24,2015-03-07 21:53,"sweet, now we can ask angular2 questions in re...",1,"['time', 'timing', 'real-time', 'angular']","sweet, now we can ask angular2 question in rea..."
2,3,530cd1eb5e986b0712efb4d2,2015-03-07 22:04,Yes!\n,1,[],ye!\n
3,4,54e6ee3b15522ed4b3dc3c24,2015-03-07 22:06,we're missing the [View Source] [Improve this ...,1,"['button', 'io', 'views', 'angular', 'ios', 'v...",we're miss the [view sourc] [improv thi doc] b...
4,5,530cd1eb5e986b0712efb4d2,2015-03-07 23:18,Good idea can you file an issue with repo?\n,1,['file'],good idea can you file an issu with repo?\n


In [8]:
len(df_issue)

422974

In [42]:
df_selected = df_issue[df_issue["tags"] != "[]"]
df_selected.reset_index(drop=True,inplace=True)

In [43]:
len(df_selected)

378280

In [44]:
df_selected.head()

,Id,asker,time,content,lines,tags,content_stem
0,2,54e6ee3b15522ed4b3dc3c24,2015-03-07 21:53,"sweet, now we can ask angular2 questions in re...",1,"['time', 'timing', 'real-time', 'angular']","sweet, now we can ask angular2 question in rea..."
1,4,54e6ee3b15522ed4b3dc3c24,2015-03-07 22:06,we're missing the [View Source] [Improve this ...,1,"['button', 'io', 'views', 'angular', 'ios', 'v...",we're miss the [view sourc] [improv thi doc] b...
2,5,530cd1eb5e986b0712efb4d2,2015-03-07 23:18,Good idea can you file an issue with repo?\n,1,['file'],good idea can you file an issu with repo?\n
3,6,54e6ee3b15522ed4b3dc3c24,2015-03-08 00:20,I submitted it an issue about view source etc ...,1,"['node-modules', 'views', 'get', 'one-to-one',...",i submit it an issu about view sourc etc on an...
4,7,54f9f23015522ed4b3dce8b5,2015-03-08 05:45,"Great job on Angular 2 so far! IMHO, the Angul...",1,"['http', 'rebase', 'typescript-generics', 'gen...","great job on angular 2 so far! imho, the angul..."


## Preprocessing

In [64]:
def str_clean(str_list):
    str_list = str_list.replace('[','')
    str_list = str_list.replace(']','')
    str_list = str_list.replace('\'','')
    
    return str_list

In [66]:
msg = []
for i in range(len(df_selected)):
    str_tag = str_clean(df_selected.iloc[i]["tags"])
    str_msg = str(df_selected.iloc[i]["content_stem"]).replace('\n','')
    data = str_msg + " " + str_tag
    msg.append(data)

In [87]:
msg = []
for i in range(len(df_selected)):
    str_tag = str_clean(df_selected.iloc[i]["tags"])
    #str_msg = str(df_selected.iloc[i]["content_stem"]).replace('\n','')
    #data = str_msg + " " + str_tag
    msg.append(str_tag)

In [88]:
len(msg)

378280

In [89]:
msg[0]

'time, timing, real-time, angular'

In [95]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(msg)

In [96]:
print(X.shape)

(378280, 2922)


In [97]:
X_train = np.array(X)

In [107]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=1000, random_state=0).fit(X)

In [108]:
kmeans.labels_

array([ 93,  98, 249, ..., 117, 513, 504])

In [109]:
len(kmeans.labels_)

378280

In [111]:
df_selected["label"]=list(kmeans.labels_)

<ipython-input-111-591c60bf4ac8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["label"]=list(kmeans.labels_)


In [112]:
df_selected.head()

,Id,asker,time,content,lines,tags,content_stem,label
0,2,54e6ee3b15522ed4b3dc3c24,2015-03-07 21:53,"sweet, now we can ask angular2 questions in re...",1,"['time', 'timing', 'real-time', 'angular']","sweet, now we can ask angular2 question in rea...",93
1,4,54e6ee3b15522ed4b3dc3c24,2015-03-07 22:06,we're missing the [View Source] [Improve this ...,1,"['button', 'io', 'views', 'angular', 'ios', 'v...",we're miss the [view sourc] [improv thi doc] b...,98
2,5,530cd1eb5e986b0712efb4d2,2015-03-07 23:18,Good idea can you file an issue with repo?\n,1,['file'],good idea can you file an issu with repo?\n,249
3,6,54e6ee3b15522ed4b3dc3c24,2015-03-08 00:20,I submitted it an issue about view source etc ...,1,"['node-modules', 'views', 'get', 'one-to-one',...",i submit it an issu about view sourc etc on an...,21
4,7,54f9f23015522ed4b3dce8b5,2015-03-08 05:45,"Great job on Angular 2 so far! IMHO, the Angul...",1,"['http', 'rebase', 'typescript-generics', 'gen...","great job on angular 2 so far! imho, the angul...",411


In [113]:
df_selected.to_csv(channel2+'/'+channel2+'_predicted.csv',index = None)